*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-07-27 12:38:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  82.3MB/s    in 0.3s    

2022-07-27 12:38:59 (82.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

Exploring the data

In [ ]:
#Look at the shape of df_ratings and the number of fields it have
print(df_ratings.shape)
print(list(df_ratings.columns))
df_ratings.head()

In [ ]:
#Look at the kind of destribution we are working with
plt.rc("font", size=15)
df_ratings.rating.value_counts(sort=False).plot(kind='bar')
plt.title('Rating Distribution\n')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.savefig('system1.png', bbox_inches='tight')
plt.show()

In [ ]:
print(df_books.shape)
print(list(df_books.columns))
df_books.head()

In [ ]:
#Show the top books that has the highest number of rating
rating_count = pd.DataFrame(df_ratings.groupby('isbn')['rating'].count())
rating_count.sort_values('rating', ascending=False).head()

,rating
isbn,
0971880107,2502
0316666343,1295
0385504209,883
0060928336,732
0312195516,723


In [ ]:
#Look at the average rating score for the books with highest number of rating
average_rating = pd.DataFrame(df_ratings.groupby('isbn')['rating'].mean())
average_rating['ratingCount'] = pd.DataFrame(df_ratings.groupby('isbn')['rating'].count())
average_rating.sort_values('ratingCount', ascending=False).head()

,rating,ratingCount
isbn,,
0971880107,1.019584,2502
0316666343,4.468726,1295
0385504209,4.652322,883
0060928336,3.448087,732
0312195516,4.334716,723


Cleaning the data

In [ ]:
"""
2 goals in cleaning the data to ensure statitical significance
Goal 1: Excludes users that has given less than 200 ratings. This is done first.
Goal 2: Excludes books that recieved less than 100 rating, inclusive of goal 1.
"""
#count the number of rating each user has given
user_RatingCount = (df_ratings.
     groupby('user')['rating'].
     count().
     reset_index().#reset the index
     rename(columns = {'rating':'userTotalRatingCount'})
    )

#count the number of rating each book has recieved
book_RatingCount = (df_ratings.
     groupby('isbn')['rating'].
     count().
     reset_index().#reset the index
     rename(columns = {'rating':'bookTotalRatingCount'})
    )

#merge the count of user and book as a new column in df_ratings
df_ratings = df_ratings.merge(user_RatingCount,how='left', 
                              left_on='user', right_on='user')
df_ratings = df_ratings.merge(book_RatingCount, how='left', 
                              left_on='isbn', right_on='isbn')

#Get rid of users that has give less than 200 rating
#Get rid of book that has recieved less than 100 rating
df_ratings_2 =df_ratings.loc[(df_ratings['userTotalRatingCount']>=200) & 
                             (df_ratings['bookTotalRatingCount']>=100)]

In [ ]:
"""
Currently, df_ratings_2 contains the relationship between user and the book.
And df_books contain detail about the book only. A new df needs to be created
that contains both df's information
"""
#Join df_ratings_2 and df_books where isbn is the same
books_with_ratings = pd.merge(df_ratings_2, df_books, on='isbn')

#If a user have rated a book multiple time, drop that duplicate
books_with_ratings_2 = books_with_ratings.drop_duplicates(['title', 'user'])

In [ ]:
"""
Inorder to train the model, a matrix needs to be created such that the row 
contains the title of the book, the column is the user, and the values of each
slot is the rating of the book given by the user. The matrix will be sparsed 
since not evert user has rated a book
"""
#Develop a table from the books_with_ratings_2 dataframe with the description
books_with_ratings_pivot = books_with_ratings_2.pivot(
    values='rating', index='title', columns='user').fillna(0)

#Turn the table into a maxtrix.
book_with_ratings_matrix = csr_matrix(books_with_ratings_pivot.values)

In [ ]:
"""
Now we need to develop the model to and train it
"""
model_knn = NearestNeighbors(algorithm='auto', metric='cosine')
model_knn.fit(book_with_ratings_matrix)
NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

NearestNeighbors(metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  """
  returns the top few similar recommended books that the user has input
  """
  #Find the book the player is refering to.
  X = books_with_ratings_pivot[books_with_ratings_pivot.index == book]
  
  #turn our value into a numpy array in the shape of (1, auto)
  X = X.to_numpy().reshape(1,-1)

  #get the distance and index between the book and the recommendation
  distances, indices = model_knn.kneighbors(X,n_neighbors=8)
  recommended_books = []
  for x in reversed(range(1,6)):
      #flatten indices and distance into one list, and find the index 
      #information of each recommendation generated.
      bookrecommended = [books_with_ratings_pivot.index
                         [indices.flatten()[x]], distances.flatten()[x]]
      recommended_books.append(bookrecommended)
  recommended_books = [book, recommended_books]
  
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
